In [1]:
import requests
import pandas as pd
import time
import math

API_KEYS = [
    "468fd187-9460-42b3-adcb-934315fb3c60", 
    "f68ae405-a48a-432d-a4f8-1847ee3afd3c",
    "69969001-6dea-4fa0-8490-533cff7aabcf",
    "66715fc5-9def-486c-8cb7-c6c943833fd1",
    "4614076f-20c1-4632-941c-8a47dec5d386",
    "ba1e9466-0f1e-482a-a711-1c2ef8f455fd",
    "ba1e9466-0f1e-482a-a711-1c2ef8f455fd",
    "06b3535f-186f-4cb7-be6d-14ae08e4b48b"
]

SAVE_INTERVAL = 150  # save every 150 processed rows


def check_number_status(phone_number, api_key, timeout=60):
    url = "https://api.starsender.online/api/check-number"
    headers = {
        "Content-Type": "application/json",
        "Authorization": api_key
    }
    payload = {"number": str(phone_number)}

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=timeout)
        response.raise_for_status()
        r = response.json()
        return r.get("data", {}).get("message", "No message")
    except requests.exceptions.Timeout:
        return "Timeout"
    except requests.exceptions.RequestException as e:
        return f"Error: {str(e)}"


def process_dataframe(df):
    total_rows = len(df)
    total_keys = len(API_KEYS)

    # Auto batching for API key rotation
    batch_size = math.ceil(total_rows / total_keys)

    print(f"Total rows: {total_rows}")
    print(f"Total API keys: {total_keys}")
    print(f"Auto batch size = {batch_size}\n")

    save_counter = 0  # count rows for autosave

    for i in range(total_rows):
        # Pick API key by row index
        key_index = min(i // batch_size, total_keys - 1)
        current_api_key = API_KEYS[key_index]

        phone_number = df.loc[i, "Whatsapp"]
        result = check_number_status(phone_number, current_api_key)

        df.loc[i, "validity"] = result

        print(f"Row {i+1}/{total_rows} | API {key_index+1}/{total_keys} | {phone_number} → {result}")

        time.sleep(5)
        save_counter += 1

        # Save file every 150 processed rows
        if save_counter >= SAVE_INTERVAL:
            df.to_excel("autosave_result.xlsx", index=False)
            print("💾 Autosaved last 150 rows → autosave_result.xlsx")
            save_counter = 0

    # Final save after finishing all rows
    df.to_excel("final_result.xlsx", index=False)
    print("\n🎉 Done! Saved final_result.xlsx")


# Example
# df = pd.read_excel("input.xlsx")
# process_dataframe(df)


In [2]:
# load data
database = pd.read_excel("Database.xlsx")

# load data validity (consist whatsapp number and the status registered or not)
data_with_validity = pd.read_excel("checkpoint_files/data_with_number_validity_updated.xlsx")

In [3]:
no_crm = database[database['CRM'].isna()]
uncheck = no_crm[(no_crm['validity'].isna()) & (no_crm['Source'] == 'Web Ads')].reset_index(drop=True)

In [4]:
uncheck

,Unnamed: 0,Whatsapp,Donatur,Bulan,Tahun,CRM,Source,Total,Frekuensi,klasifikasi_program,Preferensi,Label_Jam,Rata - rata,Day_Mode,Date_Category,Tahun_Pertama,Kategori,Badge,Avg Kategori,validity
0,45438,6281326223953,Ali Susilo,November,2025,NaN,Web Ads,100661,1,PALESTINA,1,11-12,100661,Sun,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,NaN
1,45403,6281265501719,Ridi ikatra,November,2025,NaN,Web Ads,100000,1,SUDAN,1,23-00,100000,Fri,Jumat,2025,Aktif,SILVER,100.000 - 1000.000,NaN
2,45293,6281915608908,Isti,November,2025,NaN,Web Ads,100331,1,SUDAN,1,06-07,100331,Sun,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,NaN
3,45292,6287731008588,Hamba Allah,November,2025,NaN,Web Ads,100223,1,SUDAN,1,13-14,100223,Sun,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,NaN
4,45291,6287729914149,Bungah,November,2025,NaN,Web Ads,253787,1,SUDAN,1,13-14,253787,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,45358,6282336513001,Darmiah Dameng,November,2025,NaN,Web Ads,100125,1,SUDAN,1,19-20,100125,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,NaN
190,45357,628211172426,Reisya adistia,November,2025,NaN,Web Ads,100303,1,SUDAN,1,10-11,100303,Sun,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,NaN
191,45356,6282111747462,Maryetti,November,2025,NaN,Web Ads,100859,1,PALESTINA,1,06-07,100859,Sun,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,NaN
192,45354,6282112855456,Luluk,November,2025,NaN,Web Ads,50965,1,PALESTINA,1,14-15,50965,Sun,Bukan Gajian,2025,Aktif,BRONZE,< 100.000,NaN


In [5]:
process_dataframe(uncheck)

Total rows: 194
Total API keys: 8
Auto batch size = 25

Row 1/194 | API 1/8 | 6281326223953 → Number registered
Row 2/194 | API 1/8 | 6281265501719 → Number registered
Row 3/194 | API 1/8 | 6281915608908 → Number registered
Row 4/194 | API 1/8 | 6287731008588 → Number registered
Row 5/194 | API 1/8 | 6287729914149 → Number registered
Row 6/194 | API 1/8 | 6282383280463 → Number registered
Row 7/194 | API 1/8 | 6287775598539 → Number registered
Row 8/194 | API 1/8 | 6287776476929 → Number not registered
Row 9/194 | API 1/8 | 628161320130 → Number registered
Row 10/194 | API 1/8 | 628161843774 → Number registered
Row 11/194 | API 1/8 | 628179009004 → Number registered
Row 12/194 | API 1/8 | 6281802743528 → Number registered
Row 13/194 | API 1/8 | 6285281114976 → Number registered
Row 14/194 | API 1/8 | 6282349500097 → Number not registered
Row 15/194 | API 1/8 | 628975359100 → Number registered
Row 16/194 | API 1/8 | 62895636841312 → Number not registered
Row 17/194 | API 1/8 | 628115055

In [6]:
# process_dataframe(uncheck)
val = pd.read_excel("final_result.xlsx")
val_1 = pd.read_excel("autosave_result.xlsx")
# val_2 = pd.read_excel("batch_2.xlsx")
# val_3 = pd.read_excel("batch_3.xlsx")

In [11]:
validity = pd.concat([val])  # , val_2, val_3])
validity

,Unnamed: 0,Whatsapp,Donatur,Bulan,Tahun,CRM,Source,Total,Frekuensi,klasifikasi_program,Preferensi,Label_Jam,Rata - rata,Day_Mode,Date_Category,Tahun_Pertama,Kategori,Badge,Avg Kategori,validity
0,45438,6281326223953,Ali Susilo,November,2025,NaN,Web Ads,100661,1,PALESTINA,1,11-12,100661,Sun,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
1,45403,6281265501719,Ridi ikatra,November,2025,NaN,Web Ads,100000,1,SUDAN,1,23-00,100000,Fri,Jumat,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
2,45293,6281915608908,Isti,November,2025,NaN,Web Ads,100331,1,SUDAN,1,06-07,100331,Sun,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
3,45292,6287731008588,Hamba Allah,November,2025,NaN,Web Ads,100223,1,SUDAN,1,13-14,100223,Sun,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
4,45291,6287729914149,Bungah,November,2025,NaN,Web Ads,253787,1,SUDAN,1,13-14,253787,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,45358,6282336513001,Darmiah Dameng,November,2025,NaN,Web Ads,100125,1,SUDAN,1,19-20,100125,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
190,45357,628211172426,Reisya adistia,November,2025,NaN,Web Ads,100303,1,SUDAN,1,10-11,100303,Sun,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number not registered
191,45356,6282111747462,Maryetti,November,2025,NaN,Web Ads,100859,1,PALESTINA,1,06-07,100859,Sun,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
192,45354,6282112855456,Luluk,November,2025,NaN,Web Ads,50965,1,PALESTINA,1,14-15,50965,Sun,Bukan Gajian,2025,Aktif,BRONZE,< 100.000,Number not registered


In [12]:
validity_update = pd.concat([data_with_validity, validity])
validity_update = validity_update.drop_duplicates(subset=['Whatsapp'], keep='last').reset_index(drop=True)

In [13]:
validity_update['validity'].value_counts(dropna=False)

validity
NaN                                                                                                                                                21533
Number registered                                                                                                                                  19246
Number not registered                                                                                                                               2633
Error: 504 Server Error: Gateway Time-out for url: https://api.starsender.online/api/check-number                                                     14
Error: 502 Server Error: Bad Gateway for url: https://api.starsender.online/api/check-number                                                           4
Error: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))        3
Timeout                                                                  

In [14]:
validity_update = validity_update[['Whatsapp', 'Donatur', 'Bulan', 'Tahun', 'CRM', 'Source',
       'Total', 'Frekuensi', 'klasifikasi_program', 'Preferensi', 'Label_Jam',
       'Rata - rata', 'Day_Mode', 'Date_Category', 'Tahun_Pertama', 'Kategori',
       'Badge', 'Avg Kategori', 'validity']]

In [15]:
validity_update.to_excel("checkpoint_files/data_with_number_validity_updated.xlsx")